In [1]:
import pandas as pd
import re
from bs4 import BeautifulSoup
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\shank\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_csv(r"C:\Users\shank\nlp_project\data\IMDB Dataset.csv")

In [3]:
df['label'] = df['sentiment'].map({'positive':1,'negative':0})

In [4]:
df.head(1)

,review,sentiment,label
0,One of the other reviewers has mentioned that ...,positive,1


In [5]:
def clean_text(text):
    text = BeautifulSoup(text, 'html.parser').get_text()
    text = re.sub(r"[^a-zA-Z]", " ", text)
    text = text.lower()
    return text

In [6]:
df['cleaned_review'] = df["review"].apply(clean_text)

C:\Users\shank\AppData\Local\Temp\ipykernel_39012\1811099635.py:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, 'html.parser').get_text()


In [7]:
df.head()

,review,sentiment,label,cleaned_review
0,One of the other reviewers has mentioned that ...,positive,1,one of the other reviewers has mentioned that ...
1,A wonderful little production. <br /><br />The...,positive,1,a wonderful little production the filming tec...
2,I thought this was a wonderful way to spend ti...,positive,1,i thought this was a wonderful way to spend ti...
3,Basically there's a family where a little boy ...,negative,0,basically there s a family where a little boy ...
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,1,petter mattei s love in the time of money is...


In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(df['cleaned_review'], df['label'], test_size = 0.2, random_state=42, stratify=df['label'])

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features=20000, ngram_range=(1,2))
X_train_vec = tfidf.fit_transform(X_train)
X_test_vec = tfidf.transform(X_test)

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,classification_report, confusion_matrix

model = LogisticRegression(max_iter=1500)
model.fit(X_train_vec, y_train)
y_pred = model.predict(X_test_vec)

In [12]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.91      0.90      0.91      5000
           1       0.90      0.91      0.91      5000

    accuracy                           0.91     10000
   macro avg       0.91      0.91      0.91     10000
weighted avg       0.91      0.91      0.91     10000



In [15]:
from sklearn.naive_bayes import GaussianNB,MultinomialNB
gnb = GaussianNB()
mnb = MultinomialNB()

mnb.fit(X_train_vec, y_train)
y_pred_mnb = mnb.predict(X_test_vec)
print(classification_report(y_test,y_pred_mnb))


              precision    recall  f1-score   support

           0       0.88      0.87      0.87      5000
           1       0.87      0.89      0.88      5000

    accuracy                           0.88     10000
   macro avg       0.88      0.88      0.88     10000
weighted avg       0.88      0.88      0.88     10000



In [24]:
import joblib
joblib.dump(model, r"C:\Users\shank\nlp_project\model\tfidf_logreg.joblib")
joblib.dump(tfidf, r"C:\Users\shank\nlp_project\model\tfidf_vectorizer.joblib")


['C:\\Users\\shank\\nlp_project\\model\\tfidf_vectorizer.joblib']

In [25]:
import joblib



def predict_sentiment(review):
    cleaned = clean_text(review)
    vec = vectorizer.transform([cleaned])
    pred = model.predict(vec)[0]
    return "Positive" if pred == 1 else "Negative"


In [31]:
print(predict_sentiment("The movie was a good but bad movie"))

Negative


In [33]:
# app.py
import streamlit as st
import joblib

model = joblib.load( r"C:\Users\shank\nlp_project\model\tfidf_logreg.joblib")
vectorizer = joblib.load(r"C:\Users\shank\nlp_project\model\tfidf_vectorizer.joblib")

st.title("🎬 Movie Review Sentiment Analyzer")
review = st.text_area("Enter your movie review:")
if st.button("Predict Sentiment"):
    cleaned = clean_text(review)
    vec = vectorizer.transform([cleaned])
    pred = model.predict(vec)[0]
    st.write("Prediction:", "Positive 😊" if pred == 1 else "Negative 😞")


2025-10-26 17:41:12.551 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-26 17:41:12.579 
  command:

    streamlit run C:\ProgramData\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-10-26 17:41:12.579 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-26 17:41:12.580 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-26 17:41:12.580 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-26 17:41:12.580 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-26 17:41:12.581 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-26 17:41:12.581 Session state does not 

In [34]:
streamlit run app.py


SyntaxError: invalid syntax (507122745.py, line 1)